In [412]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [415]:
# записываем датафреймы
df_fin = pd.read_csv('financial_data.csv')
df_prolong = pd.read_csv('prolongations.csv')

# удаляем дубликаты
df_fin = df_fin.drop_duplicates()
df_prolong = df_prolong.drop_duplicates()

In [416]:
# хронология месяцев и маппинг
months_ordered = [
    'Ноябрь 2022', 'Декабрь 2022', 'Январь 2023', 'Февраль 2023',
    'Март 2023', 'Апрель 2023', 'Май 2023', 'Июнь 2023',
    'Июль 2023', 'Август 2023', 'Сентябрь 2023', 'Октябрь 2023',
    'Ноябрь 2023', 'Декабрь 2023', 'Январь 2024', 'Февраль 2024'
]
month_map = {m: i+1 for i, m in enumerate(map(str.lower, months_ordered))}

In [417]:
# обработка специальных значений
def parse_val(v):
    if pd.isna(v): return 0
    s = str(v).strip().lower()
    if s in ['в ноль', 'в ноль.']: return 0
    if s in ['стоп', 'end']: return 'STOP'
    s = s.replace('\xa0', '').replace(' ', '')
    s = s.replace(',', '.')
    
    try:
        #заменяем запятую на точку для корректного float
        return float(s)
    except ValueError:
        # чтобы явно видеть, что произошло
        print(f"⚠️ Проблема при парсинге значения: {v}")
        return 0
    
    
for m in months_ordered:
    df_fin[m] = df_fin[m].apply(parse_val)

In [421]:
# объединяем датафреймы
df = df_fin.merge(df_prolong[['id','month','AM']], on='id', how='left') \
           .rename(columns={'AM':'Manager'})

In [422]:
# добавляем новый столбец с порядковым числом месяца
df['end_period'] = df['month'].map(month_map)

In [423]:
# удаляем строки с Nan и столбец
df = df.dropna(subset=['month'])
df = df.drop('Account', axis=1)

In [428]:
# исключаем проекты со stop
month_map_ = {m: i+1 for i, m in enumerate(months_ordered)}

def has_stop(row):
    end = row['end_period']
    for m, idx in month_map_.items():
        if idx <= end and row[m] == 'STOP':
            return True
    return False

In [429]:
df = df[~df.apply(has_stop, axis=1)].copy()

In [443]:
# переводим в длинный формат
df_long = df.melt(
    id_vars=['id','Manager','end_period'],
    value_vars=months_ordered,
    var_name='ship_month',
    value_name='amount'
)

In [444]:
df_long['ship_period'] = df_long['ship_month'].map(month_map_)

In [445]:
df_long = df_long[df_long['amount'] != 'STOP']

In [446]:
df_long = df_long.copy()
df_long['amount'] = pd.to_numeric(df_long['amount'], errors='coerce')

In [448]:
# убераем проекты без единой положительной отгрузки
sums_project = df_long.groupby('id')['amount'].sum()
valid_ids = sums_project[sums_project > 0].index
df_long = df_long[df_long['id'].isin(valid_ids)]

In [449]:
df_long = df_long[df_long['Manager'] != 'без А/М']

In [451]:
# функции расчета коэффициентов для каждого менеджера по месяцам
def calc_k1_manager(df, t):
    P1 = df[df['end_period']==t-1]
    denom = P1[P1['ship_period']==t-1].groupby('Manager')['amount'].sum()
    numer = P1[P1['ship_period']==t].groupby('Manager')['amount'].sum()
    return (numer / denom).rename(f'K1_period_{t}')


def calc_k2_manager(df, t):
    P2 = df[df['end_period']==t-2]
    piv = P2.pivot_table(
        index=['id','Manager'],
        columns='ship_period', values='amount',
        aggfunc='sum', fill_value=0
    ).reset_index()
    P2a = piv[piv.get(t-1, 0) == 0]
    denom = P2a.groupby('Manager')[t-2].sum()
    numer = P2a.groupby('Manager')[t].sum()
    return (numer / denom).rename(f'K2_period_{t}')

In [454]:
results_manager = []
for t in range(3, 15):  # периоды 3..14
    k1 = calc_k1_manager(df_long, t)
    k2 = calc_k2_manager(df_long, t)
    month_period = next((k.split()[0] for k, v in month_map.items() if v == t), None)
    tmp = pd.concat([k1, k2], axis=1).reset_index().assign(period=month_period)
    results_manager.append(tmp)

In [456]:
# функции расчета коэффициентов для всего отдела за каждый месяц
def calc_k1_month(df, t):
    P1 = df[df['end_period']==t-1]
    denom = P1[P1['ship_period']==t-1].groupby('ship_month')['amount'].sum().iloc[0]
    numer = P1[P1['ship_period']==t].groupby('ship_month')['amount'].sum().iloc[0]
    return pd.Series(numer / denom if denom != 0 else None).rename(f'K1_period_{t}')


def calc_k2_month(df, t):
    P2 = df[df['end_period']==t-2]
    piv = P2.pivot_table(
        index=['id'],
        columns='ship_period', values='amount',
        aggfunc='sum', fill_value=0
    ).reset_index()
    P2a = piv[piv.get(t-1, 0) == 0]
    denom = P2a[t-2].sum()
    numer = P2a[t].sum()
    return pd.Series(numer / denom if denom != 0 else None).rename(f'K2_period_{t}')

In [457]:
results_month = []
for t in range(3, 15):  # периоды 3..14
    k1 = calc_k1_m(df_long, t)
    k2 = calc_k2_m(df_long, t)
    month_period = next((k.split()[0] for k, v in month_map.items() if v == t), None)
    tmp = pd.concat([k1, k2], axis=1).assign(period=month_period)
    results_month.append(tmp)

In [493]:
# функция объединяет список датафремов в один датафрейм
def combine_period_dfs(df_list, cols=None):
    
    # Определяем порядок месяцев, чтобы потом отсортировать по календарю
    months_order = [m.split()[0].lower() for m in months_ordered[2:14]]
    
    processed = []
    for df in df_list:
        # найдём колонки с K1 и K2 для этого df
        k1_cols = [c for c in df.columns if c.startswith('K1_period_')]
        k2_cols = [c for c in df.columns if c.startswith('K2_period_')]
        if not k1_cols or not k2_cols:
            continue  # пропускаем, если чего-то нет
        
        # оставляем только нужные колонки
        if cols == 'Manager':
            tmp = df[['Manager', 'period', k1_cols[0], k2_cols[0]]].copy()
        else:
            tmp = df[['period', k1_cols[0], k2_cols[0]]].copy()
            
        # переименуем в унифицированные
        tmp = tmp.rename(columns={
            k1_cols[0]: 'K1',
            k2_cols[0]: 'K2',
            'period': 'month'
        })
        processed.append(tmp)
    
    # объединяем всё в один DF
    result = pd.concat(processed, ignore_index=True)
    if cols == 'Manager':
        # делаем month категориальным
        result['month'] = pd.Categorical(
            result['month'],
            categories=months_order,
            ordered=True
        )
    
        # сортируем
        result = result.sort_values(by=['Manager', 'month']).reset_index(drop=True)
    
    return result

In [494]:
# датафрейм с К1 и К2 для каждого менеджера по месяцам
manager_monthly = combine_period_dfs(results_manager, 'Manager')

In [496]:
# датафрейм с К1 и К2 по всему отделу за каждый месяц
dept_monthly = combine_period_dfs(results_month)

In [497]:
# датафрейм с суммой К1 и К2 по каждому менеджеру за весь год
manager_yearly = manager_monthly.groupby('Manager', as_index=False).agg({'K1': 'sum', 'K2': 'sum'})

In [498]:
# сумма К1 и К2 по всему отделу за весь год
total_k1 = manager_monthly['K1'].sum()
total_k2 = manager_monthly['K2'].sum()

In [499]:
# быстрое сохранение в Excel

with pd.ExcelWriter('analytics_report.xlsx', engine='xlsxwriter') as writer:
    manager_monthly.to_excel(writer, sheet_name='Manager_Monthly', index=False)
    dept_monthly.to_excel(writer, sheet_name='Dept_Monthly', index=False)
    manager_yearly.to_excel(writer, sheet_name='Manager_Yearly', index=False)